In [1]:
import torch
import numpy as np
import torch.nn as nn

In [2]:
X = torch.arange(1, 101, step = 2).unsqueeze(dim=1)
Y = 0.3 + 0.7*X
X.shape

torch.Size([50, 1])

In [3]:
X_train = X[:int(0.8*len(X))]
Y_train = Y[:int(0.8*len(Y))]
X_test = X[int(0.8*len(X)):]
Y_test = Y[int(0.8*len(Y)):]

In [4]:
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(1,dtype=torch.float, requires_grad=True))
        self.bias = nn.Parameter(torch.randn(1, dtype=torch.float, requires_grad=True))

    def forward(self, x: torch.Tensor):
        return self.weights*x + self.bias

In [5]:
#main classes
#torch.nn contains:
# torch.nn.Parameter
# torch.nn.Module: base class for ALL NNs .forward() needs to be defined in child class
# torch.optim: live optimizer uses grad descent
# torch.L1Loss, torch.Adam

In [6]:
torch.manual_seed(42)
model_0 = LinearRegressionModel()
list(model_0.parameters())

[Parameter containing:
 tensor([0.3367], requires_grad=True),
 Parameter containing:
 tensor([0.1288], requires_grad=True)]

In [7]:
model_0.state_dict()

OrderedDict([('weights', tensor([0.3367])), ('bias', tensor([0.1288]))])

In [8]:
# #without inference_mode()
# y_preds = model_0(X_test)
# print(y_preds)

In [9]:
# with  inference_mode()
with torch.inference_mode():
    y_preds = model_0(X_test)

print(y_preds)
#torch.no_grad() similar stuff, but inference_mode() is faster along with other benefits


tensor([[27.4007],
        [28.0741],
        [28.7475],
        [29.4209],
        [30.0943],
        [30.7676],
        [31.4410],
        [32.1144],
        [32.7878],
        [33.4612]])


In [10]:
#setup loss and optimizer
loss_fn = nn.L1Loss()
optimizer = torch.optim.SGD(params=model_0.parameters(), lr=0.01) # or Adam

In [11]:

torch.manual_seed(42)

# Set the number of epochs (how many times the model will pass over the training data)
epochs = 100

# Create empty loss lists to track values
train_loss_values = []
test_loss_values = []
epoch_count = []

for epoch in range(epochs):
    ### Training

    # Put model in training mode (this is the default state of a model)
    model_0.train()

    # 1. Forward pass on train data using the forward() method inside
    y_pred = model_0(X_train)
    # print(y_pred)

    # 2. Calculate the loss (how different are our models predictions to the ground truth)
    loss = loss_fn(y_pred, Y_train)

    # 3. Zero grad of the optimizer
    optimizer.zero_grad()

    # 4. Loss backwards
    loss.backward()

    # 5. Progress the optimizer
    optimizer.step()

    ### Testing

    # Put the model in evaluation mode
    model_0.eval()

    with torch.inference_mode():
      # 1. Forward pass on test data
      test_pred = model_0(X_test)

      # 2. Caculate loss on test data
      test_loss = loss_fn(test_pred, Y_test.type(torch.float)) # predictions come in torch.float datatype, so comparisons need to be done with tensors of the same type

      # Print out what's happening
      if epoch % 10 == 0:
            epoch_count.append(epoch)
            train_loss_values.append(loss.detach().numpy())
            test_loss_values.append(test_loss.detach().numpy())
            print(f"Epoch: {epoch} | MAE Train Loss: {loss} | MAE Test Loss: {test_loss} ")

Epoch: 0 | MAE Train Loss: 14.703577041625977 | MAE Test Loss: 3.1409382820129395 
Epoch: 10 | MAE Train Loss: 14.298568725585938 | MAE Test Loss: 4.045952796936035 
Epoch: 20 | MAE Train Loss: 14.014566421508789 | MAE Test Loss: 4.679960250854492 
Epoch: 30 | MAE Train Loss: 13.912074089050293 | MAE Test Loss: 4.907440185546875 
Epoch: 40 | MAE Train Loss: 13.809582710266113 | MAE Test Loss: 5.13492488861084 
Epoch: 50 | MAE Train Loss: 13.7070894241333 | MAE Test Loss: 5.362410068511963 
Epoch: 60 | MAE Train Loss: 13.604596138000488 | MAE Test Loss: 5.589890480041504 
Epoch: 70 | MAE Train Loss: 13.502102851867676 | MAE Test Loss: 5.81737756729126 
Epoch: 80 | MAE Train Loss: 13.39961051940918 | MAE Test Loss: 6.044861793518066 
Epoch: 90 | MAE Train Loss: 13.297117233276367 | MAE Test Loss: 6.272341728210449 


In [12]:
    model_0.eval()

    with torch.inference_mode():
      # 1. Forward pass on test data
      test_pred = model_0(X_test)

      # 2. Caculate loss on test data
      test_loss = loss_fn(test_pred, Y_test.type(torch.float)) # predictions come in torch.float datatype, so comparisons need to be done with tensors of the same type

      # Print out what's happening
      if epoch % 10 == 0:
            epoch_count.append(epoch)
            train_loss_values.append(loss.detach().numpy())
            test_loss_values.append(test_loss.detach().numpy())
            print(f"Epoch: {epoch} | MAE Train Loss: {loss} | MAE Test Loss: {test_loss} ")

In [13]:
model_0.state_dict()

OrderedDict([('weights', tensor([0.3737])), ('bias', tensor([0.1578]))])